In [1]:
import pandas as pd
import xmltodict
import json
import os
import pickle

In [4]:
# get the contact data
# {'person1':{'person1':'n1', 'person2':'n2', 'person3':'n3'}, 'person2':{...}}

def contact(linkDict, personContact, stride, link_velocity):
    # stride = 500
    # v {0.0: {'95187201', '123413'}, 3597.0: {'95187201'}, 25549.0: {'322233201'} 
    # {time:{person}}
    for k, v in linkDict.items():
        # {0.0, 3597.0}
        times = list(v.keys())
        for t1 in range(len(times)):
            for t2 in range(1, len(times)):
                if abs(times[t2] - times[t1]) <= min(stride, link_velocity[k]["length"])/(link_velocity[k]["velocity"] * 0.5):
                    mPersons = list(linkDict[k][times[t1]])
                    cPerson = list(linkDict[k][times[t2]])
                    for p in mPersons:
                        for cp in cPerson:
                            if p == cp:
                                continue
                            if not personContact.__contains__(p):
                                personContact[p] = dict()
                                if not personContact[p].__contains__(cp):
                                    personContact[p][cp] = {times[t2]}
                                else:
                                    personContact[p][cp].add(times[t2])
                            else:
                                if not personContact[p].__contains__(cp):
                                    personContact[p][cp] = {times[t2]}
                                else:
                                    personContact[p][cp].add(times[t2])
                            if not personContact.__contains__(cp):
                                personContact[cp] = dict()
                                if not personContact[cp].__contains__(p):
                                    personContact[cp][p] = {times[t2]}
                                else:
                                    personContact[cp][p].add(times[t2])
                            else:
                                if not personContact[cp].__contains__(p):
                                    personContact[cp][p] = {times[t2]}
                                else:
                                    personContact[cp][p].add(times[t2])
                            
                else:
                    break
    return personContact



In [2]:
# get the contact data
# {'person1':{'person1', 'person2', 'person3'}, 'person2':{...}}

def contact2(linkDict, personContact, stride, link_velocity):
    # stride = 500
    # v {'95187201', '123413', '322233201'} 
    for k, v in linkDict.items():
        # {0.0, 3597.0}
        times = list(v.keys())
        for t1 in range(len(times)):
            for t2 in range(1, len(times)):
                if abs(times[t2] - times[t1]) <= min(stride, link_velocity[k]["length"])/(link_velocity[k]["velocity"] * 0.5):
                    mPersons = list(linkDict[k][times[t1]])
                    cPerson = list(linkDict[k][times[t2]])
                    for p in mPersons:
                        for cp in cPerson:
                            if p == cp:
                                continue
                            if not personContact.__contains__(p):
                                personContact[p] = {cp}
                            else:
                                personContact[p].add(cp)
                            if not personContact.__contains__(cp):
                                personContact[cp] = {p}
                            else:
                                personContact[cp].add(p)
                            
                else:
                    break
    return personContact



In [5]:
with open("../network/network.xml", 'r') as fn:
    network_xml = fn.read()
    network = xmltodict.parse(network_xml, encoding='utf-8')

In [6]:


link_velocity = dict()
for link in network["network"]["links"]["link"]:
    id = link["@id"]
    speed = float(link["@freespeed"])
    length = float(link["@length"])
    link_velocity[id] = {"velocity": speed, "length":length}

In [7]:
dir = "../event_sorted_by_link/"
link_pkl_list = os.listdir(dir)

In [8]:
if link_pkl_list.__contains__('.DS_Store'):
    link_pkl_list.remove('.DS_Store')

In [12]:
strides = [i for i in range(150, 200, 10)]
strides.extend([i for i in range(200, 501, 50)])

In [13]:
strides

[150, 160, 170, 180, 190, 200, 250, 300, 350, 400, 450, 500]

In [14]:
for stride in strides:
    personContact = dict()
    idx = 0
    print('begin to load data')
    for file in link_pkl_list:
        with open(dir + file, "rb") as fr:
            linkDict = pickle.load(fr)
        # print("begin to get contact data")
        personContact = contact(linkDict, personContact, stride, link_velocity)
        # print("get the contact data in link " + str(idx))
        idx += 1
    num = personContact.__len__()
    with open("../contact_data/contact/contact_data_" + str(stride) + "_" + str(num)+ ".pkl", 'wb') as fwb:
        pickle.dump(personContact, fwb)
    
    

begin to load data
begin to load data
begin to load data
begin to load data
begin to load data
begin to load data
begin to load data
begin to load data
begin to load data
begin to load data
begin to load data
begin to load data


In [15]:
list(personContact.keys())[:10]

['6057201',
 '6184901',
 '212100901',
 '193308901',
 '357587601',
 '3482501',
 '5179901',
 '182361501',
 '270014301',
 '7078201']

In [16]:
personContact['6184901']

{'6057201': {59970.0}}

In [17]:
personContact['193308901']

{'212100901': {22230.0}}

In [18]:
len(personContact['193308901'])

1